In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

data = pd.read_csv("cleaned_input/train_consolidated.csv")

In [2]:
data.head()
data = data.drop(["msno", "Unnamed: 0"], axis=1)
data.shape

(504818, 124)

In [3]:
#data = pd.DataFrame([[1,2,3,5,6,7,8,9],[1,3,9,7,5,4,5,7]], columns=['a','b','c','d','e','f','g','is_churn'])

In [4]:
model_data = data.sample(frac=.1)

In [5]:
X_train = model_data.drop(['is_churn'], axis=1);
y_train = model_data['is_churn'];

In [6]:
X_train.isnull().any()

trans_count                    False
date_min                       False
date_max                       False
date_count                     False
num_25_sum                     False
num_50_sum                     False
num_75_sum                     False
num_985_sum                    False
num_100_sum                    False
num_unq_sum                    False
total_secs_sum                 False
Unnamed: 0.1.1                  True
city                           False
bd                             False
registered_via                 False
registration_init_year         False
registration_init_month        False
registration_init_date         False
payment_method_id              False
payment_plan_days              False
plan_list_price                False
actual_amount_paid             False
is_auto_renew                  False
is_cancel                      False
discount                       False
is_discount                    False
amt_per_day                    False
m

In [7]:
print(np.all(np.isfinite(X_train)))
print(np.any(np.isnan(X_train)))

False
True


In [8]:
def plot_learning_curve(est, X_train, y_train):
    train_size, train_score, CV_score = learning_curve(est, X_train, y_train, scoring="neg_log_loss", train_sizes=np.linspace(0.1,1, 5));
    plt.figure();
    plt.plot(train_size, np.mean(train_score, axis=1), 'o-', color='b', label='training score');
    plt.plot(train_size, np.mean(CV_score, axis=1), 'o-', color='r', label='CV score');
    plt.xlabel("Training examples");
    plt.ylabel("Score");
    plt.fill_between(train_size, np.mean(train_score, axis=1) - np.std(train_score, axis=1),
                     np.mean(train_score, axis=1) + np.std(train_score, axis=1), alpha=0.1,
                     color="b");
    plt.fill_between(train_size, np.mean(CV_score, axis=1) - np.std(CV_score, axis=1),
                     np.mean(CV_score, axis=1) + np.std(CV_score, axis=1), alpha=0.1,
                     color="r");
    plt.title("Learning Curve", fontsize=20);
    plt.legend();
    plt.show();

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [18]:
#### making list of parameters for grid search
param_logit = {'C':[0.01,0.1,1,10,100], 'penalty':['l1','l2']}
param_knn = {'n_neighbors':[1,3,6,10], 'weights':['uniform','distance']}
param_svc = {'C':[0.01,0.1,1,10,100], 'kernel':{'rbf','sigmoid','precomputed'}}
#param_ann = {hidden_layer_sizes}
param_xgb = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2),
    'min_child_weight':[6,8,10,12],
    'gamma':[i/10.0 for i in range(0,5)],
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
    }
param_lgbm = {
    'learning_rate': [0.005, .05, .1, .5],
    'n_estimators': [8,16,24, 50, 100, 150],
    'num_leaves': [6,8,12,16, 32, 64],
    'max_depth': [-1, 10, 20, 50, 100, 200],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'colsample_bytree' : [0.6, 0.8, 0.9, 1],
    'subsample' : [0.5, 0.7,1],
    'reg_alpha' : [0, 0.5, 1, 1.2, 1.4],
    'reg_lambda' : [0, 0.5, 1 ,1.2, 1.4],
}

param_rfc = {'n_estimator':[5,10,15,20], 'max_features':[10,100,500,1000]}


In [19]:
model_list = {LogisticRegression():param_logit, KNeighborsClassifier():param_knn,
             SVC():param_svc, RandomForestClassifier():param_rfc};
model_list = {LogisticRegression():param_logit}
model_list = {XGBClassifier(n_jobs=3, early_stopping_rounds=10): param_xgb}
model_list = {LGBMClassifier(nthread=3): param_lgbm}

In [ ]:
for model,param in model_list.items():
    gscv = GridSearchCV(model, param);
    print("learning curve for\t{}".format(model))
    plot_learning_curve(gscv, X_train, y_train)
    
    # summarize results
    print("Best: %f using %s" % (gscv.best_score_, gscv.best_params_))
    means = gscv.cv_results_['mean_test_score']
    stds = gscv.cv_results_['std_test_score']
    params = gscv.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))


learning curve for	LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=10, nthread=3,
        num_leaves=31, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=0, silent=True, subsample=1, subsample_for_bin=50000,
        subsample_freq=1)
